In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from metrics import dice_loss, dice_coef, iou

In [2]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [3]:
""" Global parameters """
H = 512
W = 512

In [4]:
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    masks1 = sorted(glob(os.path.join(path, "ManualMask", "rightMask", "*.png")))
 

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y1, valid_y1 = train_test_split(masks1, test_size=split_size, random_state=42)
 

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y1, test_y1 = train_test_split(train_y1, test_size=split_size, random_state=42)
 

    return (train_x, train_y1 ), (valid_x, valid_y1 ), (test_x, test_y1 )

In [6]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [7]:
def read_mask(path1):
    x1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    
    x = x1
    x = cv2.resize(x, (W, H))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [8]:
def tf_parse(x, y1 ):
    def _parse(x, y1 ):
        x = x.decode()
        y1 = y1.decode()
         

        x = read_image(x)
        y = read_mask(y1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y1], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [9]:
def tf_dataset(X, Y1, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y1))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset

In [10]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
""" Directory for storing files """
create_dir("RightMaskUPTfiles")

In [12]:
""" Hyperparameters """
batch_size = 2
lr = 1e-5
num_epochs = 30
model_path = os.path.join("RightMaskUPTfiles", "model.h5")
csv_path = os.path.join("RightMaskUPTfiles", "data.csv")

In [13]:
""" Dataset """
dataset_path = "MontgomerySet"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)} ")
print(f"Valid: {len(valid_x)} - {len(valid_y)} ")
print(f"Test: {len(test_x)} - {len(test_y)} ")


Train: 112 - 112 
Valid: 13 - 13 
Test: 13 - 13 


In [14]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

2022-09-23 08:50:13.538238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 08:50:14.714227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30624 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:65:00.0, compute capability: 7.0


## Bulid Model

In [15]:
len(train_dataset), len(valid_dataset)

(56, 7)

In [16]:

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [17]:
input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

                                                                                                  
 conv2d_8 (Conv2D)              (None, 32, 32, 1024  4719616     ['max_pooling2d_3[0][0]']        
                                )                                                                 
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 32, 32, 1024  4096       ['conv2d_8[0][0]']               
 rmalization)                   )                                                                 
                                                                                                  
 activation_8 (Activation)      (None, 32, 32, 1024  0           ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_9 

                                                                                                  
 activation_15 (Activation)     (None, 256, 256, 12  0           ['batch_normalization_15[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_transpose_3 (Conv2DTran  (None, 512, 512, 64  32832      ['activation_15[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 concatenate_3 (Concatenate)    (None, 512, 512, 12  0           ['conv2d_transpose_3[0][0]',     
                                8)                                'activation_1[0][0]']           
                                                                                                  
 conv2d_16

In [18]:
from tensorflow.keras.utils import CustomObjectScope

In [19]:
#with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]

In [20]:
mse_loss = tf.keras.losses.MeanSquaredError()

In [21]:
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)


In [22]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path)
]


In [23]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

Epoch 1/30


2022-09-23 08:50:19.877773: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


56/56 [==============================] - ETA: 0s - loss: 0.7348 - dice_coef: 0.2652 - iou: 0.1538 - recall: 0.8477 - precision: 0.2112
Epoch 1: val_loss improved from inf to 0.80197, saving model to RightMaskUPTfiles/model.h5
56/56 [==============================] - 42s 575ms/step - loss: 0.7348 - dice_coef: 0.2652 - iou: 0.1538 - recall: 0.8477 - precision: 0.2112 - val_loss: 0.8020 - val_dice_coef: 0.1935 - val_iou: 0.1077 - val_recall: 0.9812 - val_precision: 0.1353 - lr: 1.0000e-05
Epoch 2/30
56/56 [==============================] - ETA: 0s - loss: 0.6205 - dice_coef: 0.3795 - iou: 0.2361 - recall: 0.9507 - precision: 0.2994
Epoch 2: val_loss improved from 0.80197 to 0.79768, saving model to RightMaskUPTfiles/model.h5
56/56 [==============================] - 33s 587ms/step - loss: 0.6205 - dice_coef: 0.3795 - iou: 0.2361 - recall: 0.9507 - precision: 0.2994 - val_loss: 0.7977 - val_dice_coef: 0.1985 - val_iou: 0.1107 - val_recall: 0.9496 - val_precision: 0.1492 - lr: 1.0000e-05
Epo

Epoch 18/30
56/56 [==============================] - ETA: 0s - loss: 0.2705 - dice_coef: 0.7295 - iou: 0.5771 - recall: 0.9879 - precision: 0.9566
Epoch 18: val_loss did not improve from 0.29430
56/56 [==============================] - 32s 571ms/step - loss: 0.2705 - dice_coef: 0.7295 - iou: 0.5771 - recall: 0.9879 - precision: 0.9566 - val_loss: 0.3003 - val_dice_coef: 0.7020 - val_iou: 0.5467 - val_recall: 0.9301 - val_precision: 0.9462 - lr: 1.0000e-06
Epoch 19/30
56/56 [==============================] - ETA: 0s - loss: 0.2661 - dice_coef: 0.7339 - iou: 0.5813 - recall: 0.9868 - precision: 0.9611
Epoch 19: val_loss did not improve from 0.29430
56/56 [==============================] - 32s 567ms/step - loss: 0.2661 - dice_coef: 0.7339 - iou: 0.5813 - recall: 0.9868 - precision: 0.9611 - val_loss: 0.2994 - val_dice_coef: 0.7054 - val_iou: 0.5510 - val_recall: 0.9291 - val_precision: 0.9481 - lr: 1.0000e-06
Epoch 20/30
56/56 [==============================] - ETA: 0s - loss: 0.2655 - di

In [24]:
""" Predicting the mask """
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracing the image name. """
    image_name = x.split("/")[-1]

    """ Reading the image """
    ori_x = cv2.imread(x, cv2.IMREAD_COLOR)
    ori_x = cv2.resize(ori_x, (W, H))
    x = ori_x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)

    """ Reading the mask """
    ori_y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    ori_y = cv2.resize(ori_y, (W, H))
    ori_y = np.expand_dims(ori_y, axis=-1)  ## (512, 512, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (512, 512, 3)

    """ Predicting the mask. """
    y_pred = model.predict(x)[0] > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the predicted mask along with the image and GT """
    save_image_path = f"RightMaskUPTfiles/{image_name}"
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

    sep_line = np.ones((H, 10, 3)) * 255

    cat_image = np.concatenate([ori_x, sep_line, ori_y, sep_line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_image)

  0%|                                                                                                                | 0/13 [00:00<?, ?it/s]

1/1 [==============================] - 0s 290ms/step


  8%|████████                                                                                                | 1/13 [00:00<00:06,  1.80it/s]

1/1 [==============================] - 0s 18ms/step


 15%|████████████████                                                                                        | 2/13 [00:00<00:04,  2.50it/s]

...

 85%|███████████████████████████████████████████████████████████████████████████████████████▏               | 11/13 [00:03<00:00,  3.23it/s]

1/1 [==============================] - 0s 20ms/step


 92%|███████████████████████████████████████████████████████████████████████████████████████████████        | 12/13 [00:03<00:00,  3.21it/s]

1/1 [==============================] - 0s 20ms/step


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  3.01it/s]
